<a id='BacktoTop'></a>

### SMU NLP Course (DS7337) -- Homework 06 submission  (07-15-2019)
### Shanqing Gu (Class 401)

###  Table of Contents 

#### 1. [Libraries for HW6](#L)
#### 2. [Functions for HW6](#F)

#### Q1. [Evaluate text similarity of Amazon book search results](#Q1) 
 * [Q1a - Manually get 24 book titles from Amazon books with "Harry Potter"](#Q1a)
 * [Q1b - Calculate cosine similarity](#Q1b)
 * [Q1c - Rank 24 results with similarity and dissimilarity](#Q1c)

#### Q2. [Evaluate using a major search engine](#Q2)

 * [Q2a - Google book tilte 1 and copy two organic results](#Q2a)
 * [Q2b - Run the same text similarity calculation s in Q1b](#Q2b)
 * [Q2c - Which one has the highest similarity measure](#Q2c)
 
#### [Summary](#S)

<a id='L'></a>
#### 1. Libraries for HW6

[Back to top](#BacktoTop)

In [1]:
## Import libraries needed for HW5

import pandas as pd
import numpy as np
import re
import string
import nltk


from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from pattern.en import parsetree, Chunk 
from contractions import CONTRACTION_MAP
from html.parser import HTMLParser
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pattern.en import tag
from nltk.corpus import wordnet as wn

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

<a id='F'></a>
#### 2. Functions for HW6

[Back to top](#BacktoTop)

In [2]:
## expand_contractions function to expand contractions
def expand_contractions(text, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                               if contraction_mapping.get(match)\
                               else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [3]:
# annotate text tokens with POS tags
def pos_tag_text(text):
    
    def penn_to_wn_tags(pos_tag): 
        if pos_tag.startswith('J'): 
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None
    
    tagged_text = tag(text)
    tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag)) for word, pos_tag in tagged_text]
    return tagged_lower_text

In [4]:
## lemmatize text based on POS tags
def lemmatize_text(text):    
    wnl = WordNetLemmatizer()    
    pos_tagged_text = pos_tag_text(text)
    lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag else word for word, pos_tag in pos_tagged_text]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text    

In [5]:
## remove_special_characters function to remove special characters
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub(' ', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [6]:
## remove_stopwords function to remove stop words
def remove_stopwords(text):
    tokens = tokenize_text(text)
    stopword_list = nltk.corpus.stopwords.words('english')
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [7]:
## keep_text_characters to keep text characters
def keep_text_characters(text):
    filtered_tokens = []
    tokens = tokenize_text(text)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [8]:
## tokenize_text function for tokenization
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

In [9]:
## normalize_corpus function to normalize corpus
def normalize_corpus(corpus, lemmatize=True, only_text_chars=False, tokenize=False):
    
    normalized_corpus = []
    html_parser = HTMLParser()
    
    for text in corpus:
        text = html_parser.unescape(text)        
        text = expand_contractions(text, CONTRACTION_MAP)
        
        if lemmatize:
            text = lemmatize_text(text)
        else:
            text = text.lower()
        text = remove_special_characters(text)
        text = remove_stopwords(text)
        
        if only_text_chars:
            text = keep_text_characters(text)
        
        if tokenize:
            text = tokenize_text(text)
            normalized_corpus.append(text)
        else:
            normalized_corpus.append(text)
            
    return normalized_corpus

<a id='Q1'></a>
#### Q1: Evaluate text similarity of Amazon book search results by doing the following:

a.	Do a book search on Amazon. Manually copy the full book title (including subtitle) of each of the top 24 books listed in the first two pages of search results. 

b.	In Python, run one of the text-similarity measures covered in this course, e.g., cosine similarity. Compare each of the book titles, pairwise, to every other one. 

c.	Which two titles are the most similar to each other? Which are the most dissimilar? Where do they rank, among the first 24 results?

[Back to top](#BacktoTop)

<a id='Q1a'></a>
#### Q1a : First 24 book titles

Do a book search on Amazon. Manually copy the full book title (including subtitle) of each of the top 24 books listed in the first two pages of search results. 

[Back to top](#BacktoTop)

In [10]:
## get 24 book titles from Amazon books with "Harry Potter"
corpus = [
    "The Unofficial Harry Potter Spellbook: Wizard Training: Black and White Version",
    "155 Harry Potter Facts: The Ultimate Trivia Book for Wizards and Witches",
    "Harry Potter and the Goblet of Fire: The Illustrated Edition",
    "Harry Potter and the Chamber of Secrets",
    "Harry Potter and the Sorcerer's Stone, Book 1",
    "Harry Potter and the Order of the Phoenix, Book 5",
    "Harry Potter and the Deathly Hallows, Book 7",
    "Harry Potter and the Goblet of Fire, Book 4",
    "Harry Potter and the Half-Blood Prince, Book 6",
    "Harry Potter Complete Book Series Special Edition Boxed Set",
    "Harry Potter Word Search: Find over 1,600 words from J.K Rowling’s magical books and films including Hogwarts, the characters you love, spells, actors and more in this unofficial Puzzle Book",
    "Diary of a Wimpy Harry Potter: The Cowardly Wizard: Hillarious Story Of a Wimppy Harry Potter (Perfect For Kids Ages 9-12)",
    "Harry Potter Coloring Book",
    "Harry Potter: Collectible Quidditch Set - Accessory",
    "Harry Potter and the Prisoner of Azkaban, Book 3",
    "Harry Potter Origami: Fifteen Paper-Folding Projects Straight from the Wizarding World! (Harry Potter)",
    "Harry Potter Paperback Box Set (Books 1-7)",
    "Harry Potter The Illustrated Collection J.K.Rowling 3 Books Classics NEW",
    "The Unofficial Harry Potter Cookbook: From Cauldron Cakes to Knickerbocker Glory--More Than 150 Magical Recipes for Wizards and Non-Wizards Alike (Unofficial Cookbook)",
    "Harry Potter Hard Cover Boxed Set: Books #1-7",
    "Diary of a Wimpy Harry Potter: Toilet Town Monkey Trouble: Hillarious Story Of a Wimppy Harry Potter (Perfect For Kids Ages 9-12)",
    "Insight Editions IncrediBuilds Emblematics: Harry Potter: Hufflepuff",
    "Harry Potter and the Cursed Child, Parts 1 & 2, Special Rehearsal Edition Script",
    "Harry Potter: The Complete Collection (1-7)"
]

In [11]:
print ("There are", len(corpus), "book titles from Amazon books with 'Harry Potter'")

There are 24 book titles from Amazon books with 'Harry Potter'


<a id='Q1b'></a>
#### Q1b : Calculate cosine similarity

In Python, run one of the text-similarity measures covered in this course, e.g., cosine similarity. Compare each of the book titles, pairwise, to every other one. 

[Back to top](#BacktoTop)

In [12]:
## feature extraction function
def build_feature_matrix(documents, feature_type = 'frequency',
                         ngram_range = (1, 1), min_df = 0.0, max_df = 1.0):

    feature_type = feature_type.lower().strip()  
    
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary = True, min_df = min_df,
                                     max_df = max_df, ngram_range = ngram_range)
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary = False, min_df = min_df,
                                     max_df = max_df, ngram_range = ngram_range)
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df = min_df, max_df = max_df, 
                                     ngram_range = ngram_range)
    else:
        raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")

    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    
    return vectorizer, feature_matrix

In [13]:
## normalize and extract features
norm_corpus = normalize_corpus(corpus, lemmatize = True)
tfidf_vectorizer, tfidf_features = build_feature_matrix(norm_corpus,
                                                        feature_type = 'tfidf',
                                                        ngram_range = (1, 1),
                                                        min_df = 0.0,
                                                        max_df = 1.0)

query_docs_tfidf = tfidf_vectorizer.transform(norm_corpus)

In [14]:
## pairwise comparison of cosine similarity for each book title (just let top_n=3)
def compute_cosine_similarity(doc_features, corpus_features, top_n=3): 
    doc_features = doc_features.toarray()[0]    
    corpus_features = corpus_features.toarray() # get document vectors  
       
    similarity = np.dot(doc_features, corpus_features.T) # compute similarities 
       
    top_docs = similarity.argsort()[::-1][:top_n]  # get docs with highest similarity scores   
    top_docs_with_score = [(index, round(similarity[index], 3)) for index in top_docs]    
    
    bottom_docs = similarity.argsort()[::1][:top_n] # get docs with lowest similarity scores     
    bottom_docs_with_score = [(index, round(similarity[index], 3)) for index in bottom_docs] 
    
    return top_docs_with_score, bottom_docs_with_score  

In [15]:
## print the title similarity analysis using cosine similarity
print ('\nText similarity of 24 Amazon book title realted to Harry Porter using Cosine similarity measure')
print('*'*95)     
for index, doc in enumerate(corpus):
    doc_tfidf = query_docs_tfidf[index] 
    top_similar_docs, bottom_similar_docs = compute_cosine_similarity(doc_tfidf, tfidf_features, top_n=2) 
        
    print ('\n\nTitle',index,':', corpus[index])
    print ('~'*92)
    
    print ('*** Most similar title:\n')        
    for doc_index, sim_score in top_similar_docs:
        print ('Title num: {} Similarity Score: {}\nTitle: {}'. format(doc_index, sim_score, corpus[doc_index]))               
        print ('.'*34)
    
    print ('\n*** Most dissimilar title:\n' )        
    for doc_index, sim_score in bottom_similar_docs: 
        print ('Title num: {} Similarity Score: {}\nTitle: {}'. format(doc_index, sim_score, corpus[doc_index]))       
        print ('.'*34 )            


Text similarity of 24 Amazon book title realted to Harry Porter using Cosine similarity measure
***********************************************************************************************


Title 0 : The Unofficial Harry Potter Spellbook: Wizard Training: Black and White Version
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*** Most similar title:

Title num: 0 Similarity Score: 1.0
Title: The Unofficial Harry Potter Spellbook: Wizard Training: Black and White Version
..................................
Title num: 18 Similarity Score: 0.192
Title: The Unofficial Harry Potter Cookbook: From Cauldron Cakes to Knickerbocker Glory--More Than 150 Magical Recipes for Wizards and Non-Wizards Alike (Unofficial Cookbook)
..................................

*** Most dissimilar title:

Title num: 22 Similarity Score: 0.025
Title: Harry Potter and the Cursed Child, Parts 1 & 2, Special Rehearsal Edition Script
..................................
Ti

<a id='Q1c'></a>
#### Q1c : Rank 24 results with similarity and dissimilarity

Which two titles are the most similar to each other? Which are the most dissimilar? Where do they rank, among the first 24 results?

[Back to top](#BacktoTop)

#### Most similar titles (#11 and #20) with similarity Score: 0.74

#### Title 11 : 
Diary of a Wimpy Harry Potter: The Cowardly Wizard: Hillarious Story Of a Wimppy Harry Potter (Perfect For Kids Ages 9-12)

#### Title 20:
Diary of a Wimpy Harry Potter: Toilet Town Monkey Trouble: Hillarious Story Of a Wimppy Harry Potter (Perfect For Kids Ages 9-12)

#### Most dissimilar titles (#22 and #10, and #22 and #18) with similarity Score: 0.015


#### Title 22 : 
Harry Potter and the Cursed Child, Parts 1 & 2, Special Rehearsal Edition Script

#### Title 10:
Harry Potter Word Search: Find over 1,600 words from J.K Rowling’s magical books and films including Hogwarts, the characters you love, spells, actors and more in this unofficial Puzzle Book

#### Title 18:
The Unofficial Harry Potter Cookbook: From Cauldron Cakes to Knickerbocker Glory--More Than 150 Magical Recipes for Wizards and Non-Wizards Alike (Unofficial Cookbook)

<a id='Q2'></a>
#### Q2: Evaluate using a major search engine

a.	Enter one of the book titles from question 1a into Google, Bing, or Yahoo!. Copy the capsule of the first organic result and the 20th organic result. Take web results only (i.e., not video results), and skip sponsored results. 

b.	Run the same text similarity calculation that you used for question 1b on each of these capsules in comparison to the original query (book title). 

c.	Which one has the highest similarity measure? 
Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings. 

[Back to top](#BacktoTop)

<a id='Q2a'></a>
#### Q2a: Google book tilte 1 and copy two organic results

Enter one of the book titles from question 1a into Google, Bing, or Yahoo!. Copy the capsule of the first organic result and the 20th organic result. Take web results only (i.e., not video results), and skip sponsored results. 

[Back to top](#BacktoTop)

In [16]:
## Google book tilte 1: The Unofficial Harry Potter Spellbook: Wizard Training: Black and White Version" 
book_title = ['The Unofficial Harry Potter Spellbook: Wizard Training: Black and White Version']

## Capsule for the capsule of the first organic result 
result_01 = ["The Unofficial Harry Potter Spellbook: Wizard Training - Amazon.com https://www.amazon.com/Unofficial-Harry-Potter-Spellbook-Training/dp/1542855829 The Unofficial Harry Potter Spellbook: Wizard Training: Black and White Version [ Michael Gonzalez] on Amazon.com. *FREE* shipping on qualifying offers"]

## Capsule for the capsule of the 20th organic result
result_20 = ["The Unofficial Harry Potter Spellbook : Michael K Gonzalez ...https://www.bookdepository.com/Unofficial-Harry-Potter-Spellbook.../97815428558... Jan 30, 2017 - The Unofficial Harry Potter Spellbook by Michael K Gonzalez, ... Unofficial Harry Potter Spellbook : Wizard Training: Black and White Version"]
combined = book_title + result_01 + result_20

<a id='Q2b'></a>
#### Q2b: Run the same text similarity calculation s in Q1b

Run the same text similarity calculation that you used for question 1b on each of these capsules in comparison to the original query (book title).

[Back to top](#BacktoTop)

In [17]:
# normalize and extract features
norm_book_corpus = normalize_corpus(combined, lemmatize=True) 
title_book_corpus = normalize_corpus(book_title, lemmatize=True) 
tfidf_vectorizer, tfidf_features = build_feature_matrix(norm_book_corpus,                                                         
                                                        feature_type='tfidf', 
                                                        ngram_range=(1, 1), 
                                                        min_df=0.0, 
                                                        max_df=1.0)
query_docs_tfidf = tfidf_vectorizer.transform(norm_book_corpus)

In [18]:
## print the title similarity analysis using cosine similarity
print ('Compare google search for book title #1 realted to Harry Porter using Cosine similarity measure')
print('*'*95)     
for index, doc in enumerate(combined):
    doc_tfidf = query_docs_tfidf[index] 
    top_similar_docs, bottom_similar_docs = compute_cosine_similarity(doc_tfidf, tfidf_features, top_n=3) 
        
    print ('\n\nTitle',index,':', combined[index])
    print ('~'*92)
    
    print ('*** Most similar title:\n')        
    for doc_index, sim_score in top_similar_docs:
        print ('Capsule num: {} Similarity Score: {}\nCapsule: {}'. format(doc_index, sim_score, combined[doc_index]))               
        print ('.'*34)
    
    print ('\n*** Most dissimilar title:\n' )        
    for doc_index, sim_score in bottom_similar_docs: 
        print ('Capsule num: {} Similarity Score: {}\nCapsule: {}'. format(doc_index, sim_score, combined[doc_index]))       
        print ('.'*34 )

Compare google search for book title #1 realted to Harry Porter using Cosine similarity measure
***********************************************************************************************


Title 0 : The Unofficial Harry Potter Spellbook: Wizard Training: Black and White Version
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*** Most similar title:

Capsule num: 0 Similarity Score: 1.0
Capsule: The Unofficial Harry Potter Spellbook: Wizard Training: Black and White Version
..................................
Capsule num: 2 Similarity Score: 0.695
Capsule: The Unofficial Harry Potter Spellbook : Michael K Gonzalez ...https://www.bookdepository.com/Unofficial-Harry-Potter-Spellbook.../97815428558... Jan 30, 2017 - The Unofficial Harry Potter Spellbook by Michael K Gonzalez, ... Unofficial Harry Potter Spellbook : Wizard Training: Black and White Version
..................................
Capsule num: 1 Similarity Score: 0.618
Capsule: The 

<a id='Q2c'></a>
#### Q2c: Which one has the highest similarity measure?

Which one has the highest similarity measure? Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings.

[Back to top](#BacktoTop)


#### Search title #1: 

The Unofficial Harry Potter Spellbook: Wizard Training: Black and White Version

#### result_01 :

Similarity Score: 0.695

The Unofficial Harry Potter Spellbook: Wizard Training - Amazon.com

#### result_20

Similarity Score: 0.652

The Unofficial Harry Potter Spellbook : Michael K Gonzalez 

<a id='S'></a>
#### Summary

[Back to top](#BacktoTop)

In HW6, functions are defined to take in the vectorized corpus and the document corpus for computing similarities. The similarity scores are from the dot product operation and sorted them in reverse (forward) order to get the top (bottom) n documents with the highest (lowest) similarity score. 

The top search for a book title from both Amazon and Google can get high cosine similarity score. We can conclude both amazon and google use similar distance measures to rank their items online.

In addition, Scikit-learn’s cosine_similarity() utility functioncan be found in the sklearn.metrics.pairwise module. It uses similar logic and much more optimized, especially performing well on large corpora of documents. 

#### References

Sarkar D. Text analytics with Python, 2nd edition

#### Implementation environments

In [19]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import pandas;  print("Pandas", pandas.__version__)
import numpy; print("NumPy", numpy.__version__)
import requests; print("Requests", requests.__version__)
import pickle; print("Pickle", pickle.format_version)
import re; print("Re", re.__version__)
import nltk; print("nltk", nltk.__version__)
import matplotlib; print("Matplotlib", matplotlib.__version__)
import pattern.en; print("Pattern", pattern.__version__)

Darwin-18.6.0-x86_64-i386-64bit
Python 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas 0.24.2
NumPy 1.16.4
Requests 2.22.0
Pickle 4.0
Re 2.2.1
nltk 3.4
Matplotlib 3.0.2
Pattern 3.6


##### -- The End of Homework 06 --